In [1084]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [1085]:
# libraries for lemmatization
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')

from nltk.stem.porter import *
import spacy
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /Users/xinyu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [1086]:
import gensim
from gensim.utils import simple_preprocess

In [1087]:
# load the models
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
stemmer = SnowballStemmer('english')

[nltk_data] Downloading package wordnet to /Users/xinyu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/xinyu/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [1088]:
# Load the preprocessed CSV file into a Pandas DataFrame
df = pd.read_csv('/Users/xinyu/Downloads/cleaned_strings - cleaned_strings.csv',encoding='latin-1')

In [1089]:
# Overview of the document
print(df.head(2))
print(len(df))
print('Category')
print(len(df.groupby('category')))

   Unnamed: 0  category                                             string
0           0  business  quarterly profits media giant timewarner jumpe...
1           1  business  dollar hit highest level euro almost months fe...
2225
Category
5


In [1090]:
# Tokenize the text column
df['tokenized_texts'] = df['string'].apply(lambda x: simple_preprocess(x))
tokenized_texts = list(df['tokenized_texts'])
print(tokenized_texts[:1])

[['quarterly', 'profits', 'media', 'giant', 'timewarner', 'jumped', 'bn', 'months', 'december', 'yearearlier', 'firm', 'biggest', 'investors', 'google', 'benefited', 'sales', 'highspeed', 'internet', 'connections', 'higher', 'advert', 'sales', 'timewarner', 'fourth', 'quarter', 'sales', 'rose', 'bn', 'bn', 'profits', 'buoyed', 'oneoff', 'gains', 'offset', 'profit', 'dip', 'warner', 'bros', 'less', 'users', 'aol', 'warner', 'friday', 'owns', 'searchengine', 'google', 'own', 'internet', 'business', 'aol', 'mixed', 'fortunes', 'lost', 'subscribers', 'fourth', 'quarter', 'profits', 'lower', 'preceding', 'quarters', 'however', 'company', 'aols', 'underlying', 'profit', 'exceptional', 'items', 'rose', 'stronger', 'internet', 'advertising', 'revenues', 'hopes', 'increase', 'subscribers', 'offering', 'online', 'service', 'free', 'timewarner', 'internet', 'customers', 'try', 'sign', 'aols', 'existing', 'customers', 'highspeed', 'broadband', 'timewarner', 'restate', 'results', 'following', 'prob

In [1091]:
"""
# Build the bigram and trigram models

bigram = gensim.models.Phrases(tokenized_texts, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[tokenized_texts], threshold=50)  


# See trigram example
print(trigram_mod[bigram_mod[tokenized_texts[0]]])
"""

'\n# Build the bigram and trigram models\n\nbigram = gensim.models.Phrases(tokenized_texts, min_count=5, threshold=50) # higher threshold fewer phrases.\ntrigram = gensim.models.Phrases(bigram[tokenized_texts], threshold=50)  \n\n\n# See trigram example\nprint(trigram_mod[bigram_mod[tokenized_texts[0]]])\n'

In [1092]:
"""
# Define functions for bigrams, trigrams and lemmatization
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out
"""

'\n# Define functions for bigrams, trigrams and lemmatization\ndef make_bigrams(texts):\n    return [bigram_mod[doc] for doc in texts]\n\ndef make_trigrams(texts):\n    return [trigram_mod[bigram_mod[doc]] for doc in texts]\n\ndef lemmatization(texts, allowed_postags=[\'NOUN\', \'ADJ\', \'VERB\']):\n    texts_out = []\n    for sent in texts:\n        doc = nlp(" ".join(sent)) \n        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])\n    return texts_out\n'

In [1093]:
"""
# Form Bigrams
data_words_bigrams = make_bigrams(tokenized_texts)

# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, vb
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'VERB', 'ADJ'])

print(data_lemmatized[:1])
"""


'\n# Form Bigrams\ndata_words_bigrams = make_bigrams(tokenized_texts)\n\n# python3 -m spacy download en\nnlp = spacy.load("en_core_web_sm", disable=[\'parser\', \'ner\'])\n\n# Do lemmatization keeping only noun, vb\ndata_lemmatized = lemmatization(data_words_bigrams, allowed_postags=[\'NOUN\', \'VERB\', \'ADJ\'])\n\nprint(data_lemmatized[:1])\n'

In [1094]:
"""
# define a function to lemmatize and stem the tokens
def lemmatize_stemming(text):
    stemmer = SnowballStemmer('english')
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
"""

"\n# define a function to lemmatize and stem the tokens\ndef lemmatize_stemming(text):\n    stemmer = SnowballStemmer('english')\n    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))\n"

In [1095]:
"""
# apply the function to the tokenized texts
df['lemmatized_stemmed_tokens'] = df['tokenized_texts'].apply(lambda x: [lemmatize_stemming(token) for token in x])
data_lemmatized = list(df['lemmatized_stemmed_tokens'])
"""

"\n# apply the function to the tokenized texts\ndf['lemmatized_stemmed_tokens'] = df['tokenized_texts'].apply(lambda x: [lemmatize_stemming(token) for token in x])\ndata_lemmatized = list(df['lemmatized_stemmed_tokens'])\n"

In [1096]:

# lemmatize the tokens
lemmatizer = WordNetLemmatizer()
df['lemmatized_tokens'] = df['tokenized_texts'].apply(lambda x: [lemmatizer.lemmatize(token) for token in x])
data_lemmatized = list(df['lemmatized_tokens'])
print(data_lemmatized[:1])

[['quarterly', 'profit', 'medium', 'giant', 'timewarner', 'jumped', 'bn', 'month', 'december', 'yearearlier', 'firm', 'biggest', 'investor', 'google', 'benefited', 'sale', 'highspeed', 'internet', 'connection', 'higher', 'advert', 'sale', 'timewarner', 'fourth', 'quarter', 'sale', 'rose', 'bn', 'bn', 'profit', 'buoyed', 'oneoff', 'gain', 'offset', 'profit', 'dip', 'warner', 'bros', 'le', 'user', 'aol', 'warner', 'friday', 'owns', 'searchengine', 'google', 'own', 'internet', 'business', 'aol', 'mixed', 'fortune', 'lost', 'subscriber', 'fourth', 'quarter', 'profit', 'lower', 'preceding', 'quarter', 'however', 'company', 'aols', 'underlying', 'profit', 'exceptional', 'item', 'rose', 'stronger', 'internet', 'advertising', 'revenue', 'hope', 'increase', 'subscriber', 'offering', 'online', 'service', 'free', 'timewarner', 'internet', 'customer', 'try', 'sign', 'aols', 'existing', 'customer', 'highspeed', 'broadband', 'timewarner', 'restate', 'result', 'following', 'probe', 'security', 'excha

In [1097]:
"""
# stem the tokens
stemmer = SnowballStemmer('english')
df['stemmed_tokens'] = df['tokenized_texts'].apply(lambda x: [stemmer.stem(token) for token in x])
"""

"\n# stem the tokens\nstemmer = SnowballStemmer('english')\ndf['stemmed_tokens'] = df['tokenized_texts'].apply(lambda x: [stemmer.stem(token) for token in x])\n"

In [1098]:

# Create a bag-of-words representation of the preprocessed documents
dictionary = gensim.corpora.Dictionary(data_lemmatized)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
corpus = [dictionary.doc2bow(doc) for doc in data_lemmatized]

In [1099]:
"""
dictionary = Dictionary(tokenized_texts)
corpus = [dictionary.doc2bow(doc) for doc in tokenized_texts]

print(corpus[:1])
"""

'\ndictionary = Dictionary(tokenized_texts)\ncorpus = [dictionary.doc2bow(doc) for doc in tokenized_texts]\n\nprint(corpus[:1])\n'

In [1100]:
# Train the LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, passes=10)

In [1101]:
# Print the topics and the most significant words for each topic (business, politics, sports, entertainment, tech):
lda_model.show_topics(5,10)

[(0,
  '0.013*"user" + 0.012*"software" + 0.010*"system" + 0.010*"firm" + 0.009*"network" + 0.009*"phone" + 0.008*"company" + 0.008*"technology" + 0.008*"service" + 0.007*"site"'),
 (1,
  '0.014*"game" + 0.011*"mobile" + 0.011*"music" + 0.010*"technology" + 0.009*"digital" + 0.009*"phone" + 0.008*"player" + 0.008*"tv" + 0.008*"service" + 0.007*"video"'),
 (2,
  '0.007*"government" + 0.007*"bn" + 0.005*"party" + 0.005*"company" + 0.005*"labour" + 0.005*"minister" + 0.005*"election" + 0.005*"country" + 0.004*"market" + 0.004*"plan"'),
 (3,
  '0.011*"game" + 0.008*"film" + 0.007*"world" + 0.005*"year" + 0.005*"play" + 0.005*"player" + 0.005*"win" + 0.005*"award" + 0.004*"england" + 0.004*"won"'),
 (4,
  '0.015*"net" + 0.013*"attack" + 0.008*"online" + 0.007*"access" + 0.006*"world" + 0.006*"security" + 0.006*"project" + 0.005*"report" + 0.005*"information" + 0.005*"government"')]

In [1102]:
# Visualize the topics
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimivis

In [1103]:
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary=dictionary)
pyLDAvis.display(vis)

/Users/xinyu/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [1104]:
# Evaluate the model
# We separate the corpus for training and testing, so that they are preprocessed in the same way.
train_corpus = corpus[:int(0.9 * len(corpus))]
test_corpus = corpus[int(0.1 * len(corpus)):]

num_topics = 5
lda_model = LdaModel(corpus=train_corpus, id2word=dictionary, num_topics=num_topics)

In [1105]:
# Evaluate the model using the testing set
perplexity = lda_model.log_perplexity(test_corpus)
print("Perplexity Score: ", perplexity)

Perplexity Score:  -7.922122444181001


In [1106]:
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.matutils import sparse2full


# perform the same preprocessing steps
testdf = pd.read_csv('/Users/xinyu/Downloads/articles_test_90_10.csv',encoding='latin-1')

testdf['tokens'] = testdf['string'].apply(lambda x: simple_preprocess(x))
tokens = list(testdf['tokens'])
data_words_bigrams = make_bigrams(tokens)
test_lem = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'VERB', 'ADJ'])

dictionary = Dictionary(test_lem)
testcorpus = [dictionary.doc2bow(doc) for doc in test_lem]

print(len(testdf.groupby('category')))
print(len(testdf))

5
223


In [1107]:
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.matutils import sparse2full


# perform the same preprocessing steps
testdf = pd.read_csv('/Users/xinyu/Downloads/articles_test_90_10.csv',encoding='latin-1')

testdf['tokens'] = testdf['string'].apply(lambda x: simple_preprocess(x))
tokens = list(testdf['tokens'])

data_words_bigrams = make_bigrams(tokens)
test_lem = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', ])

dictionary = Dictionary(test_lem)
testcorpus = [dictionary.doc2bow(doc) for doc in test_lem]

dictionary = Dictionary(test_lem)
testcorpus = [dictionary.doc2bow(doc) for doc in test_lem]
print(len(testdf.groupby('category')))
print(len(testdf))

5
223


In [1114]:
from sklearn.metrics import f1_score
"""dictionary = Dictionary(data_lemmatized)"""
dictionary = Dictionary(tokenized_texts)
num_topics = 5
lda_model = LdaModel(corpus=train_corpus, id2word=dictionary, num_topics=num_topics)


label2id = {'tech': 0, 'entertainment': 1, 'politics': 2, 'sport': 3, 'business': 4}
true_labels = testdf['category']
true_labels_num = [label2id[label] for label in true_labels]

print(true_labels_num)


[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [1115]:
"""dictionary = Dictionary(data_lemmatized)"""
dictionary = Dictionary(tokenized_texts)
num_topics = 5
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)

# Transform the preprocessed unseen text data into topic probability distributions
topic_probs = lda_model.get_document_topics(testcorpus)

# Choose the most probable topic for each document
predicted_topics = [max(tp, key=lambda x: x[1])[0] for tp in topic_probs]

# Compute the accuracy
accuracy = sum(1 for i, j in zip(predicted_topics, true_labels_num) if i == j) / len(predicted_topics)

In [1116]:
print(predicted_topics)

[4, 2, 2, 2, 2, 2, 2, 4, 4, 2, 3, 2, 2, 0, 2, 4, 2, 4, 4, 3, 3, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 4, 2, 3, 2, 2, 3, 2, 2, 2, 4, 2, 0, 4, 2, 2, 3, 0, 4, 4, 0, 2, 1, 3, 2, 2, 4, 2, 4, 1, 4, 3, 2, 3, 2, 2, 1, 3, 3, 4, 3, 4, 1, 3, 1, 4, 3, 1, 3, 3, 2, 1, 1, 2, 3, 2, 1, 3, 3, 1, 2, 2, 2, 3, 2, 2, 2, 3, 4, 2, 2, 2, 2, 2, 3, 2, 0, 2, 3, 2, 3, 2, 2, 2, 2, 3, 2, 3, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 4, 2, 2, 2, 2, 2, 3, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2]


In [1117]:
print(accuracy)

0.22869955156950672


In [1118]:
# Compute precision, recall, and F1 score
f1 = f1_score(true_labels_num, predicted_topics, average='macro')

In [1119]:
print(f1)

0.20824441170318356
